<a href="https://colab.research.google.com/github/MocT117/Another-one-/blob/master/KPI_Logistic_Process_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import plotly.graph_objects as go
import os
import pandas as pd
import config
import numpy as np

st.logo(
    image=config.horizontal,
    size = "large",
    icon_image=config.icon
    )

st.set_page_config(page_title="Semáforo EM", page_icon=":bar_chart:", layout="centered")
st.title("KPI - Logistic Process 2.")

if "df_base" in st.session_state:
    df = st.session_state["df_base"]
else:
    st.warning("Primero ve a 'Supply Chain Process - Order Management' para cargar el archivo.")

# Convertir a fechas
df['Fecha pago pedimento'] = pd.to_datetime(df['Fecha pago pedimento'], errors='coerce')
df['Fecha EM'] = pd.to_datetime(df['Fecha EM'], errors='coerce')

# Filtrar filas válidas (con ambas fechas)
df_validas = df[
    df['Fecha pago pedimento'].notna() &
    df['Fecha EM'].notna() &
    (df['Fecha pago pedimento'] >= pd.to_datetime("1971-01-01"))
    ].copy()

# Calcular diferencia en días
df_validas['Diferencia días'] = (df_validas['Fecha EM'] - df_validas['Fecha pago pedimento']).dt.days

# === EXCLUIR VALORES NEGATIVOS ===
df_validas = df_validas[df_validas['Diferencia días'] >= 0].copy()

# Semáforo: On time <= 2 días naturales
def status_semáforo(dias):
    if pd.isna(dias):
        return 'Sin fecha'
    elif dias <= 2:
        return 'On time'
    elif dias > 2:
        return 'Delay'
    else:
        return 'Revisar'

df_validas['Semáforo'] = df_validas['Diferencia días'].apply(status_semáforo)

num_filas = len(df_validas)

df_validas['mes'] = df_validas['Fecha de PO'].dt.to_period('M')

meses_opciones = ["Todos"] + [str(m) for m in sorted(df_validas['mes'].dropna().unique())]
mes_elegido = st.selectbox("📅 Selecciona un mes:", options=meses_opciones)

if mes_elegido != "Todos":
    df_filtrado = df_validas[df_validas['mes'].astype(str) == mes_elegido]
else:
    df_filtrado = df_validas

# Mostrar tabla
st.subheader("Fecha de Pago de Pedimento vs Fecha EM")
st.dataframe(df_filtrado[['PO', 'Fecha pago pedimento', 'Fecha EM', 'Diferencia días', 'Semáforo']])

# Gauge chart
st.subheader("Semáforo Fecha de Pago de Pedimento vs Fecha EM")

if not df_filtrado['Diferencia días'].dropna().empty:
    promedio_dias = df_filtrado['Diferencia días'].mean().round(0)
else:
    promedio_dias = 0

minimo = 0
maximo = max(20, int((df_filtrado['Diferencia días'].max() // 2 + 1) * 2))
tick_vals = list(range(minimo, maximo+1, 2))

fig = go.Figure(go.Indicator(
    mode = "gauge+number+delta",
    delta={'reference': 2, 'increasing': {'color': "red"}, 'decreasing': {'color': "green"}},
    value = promedio_dias,
    number = {'suffix': " días", 'font': {'size': 36}},
    gauge = {
        'axis': {'range': [minimo, maximo], 'tickmode': 'array', 'tickvals': tick_vals, 'tickfont': {'size': 16}},
        'steps': [
            {'range': [minimo, 2], 'color': 'green'},
            {'range': [2, 6], 'color': 'yellow'},
            {'range': [6, 10], 'color': 'orange'},
            {'range': [10, maximo], 'color': 'red'}
        ],
        'bar': {'color': "black", 'thickness': 0.25}
        }
    ))

st.plotly_chart(fig)
st.markdown(
    f"Promedio considerando {num_filas} diferencias (fechas confirmadas de pago de pedimento.)",
    unsafe_allow_html=True
    )

st.warning(
    f"Las diferencias negativas han sido excluidas del análisis."
)

# ==== CURVA DE PERCENTILES (1-99) ====
st.subheader("Curva de percentiles de las diferencias (días)")

# Serie de diferencias (sin nan)
serie_percentil = df_validas['Diferencia días'].dropna().sort_values().reset_index(drop=True)

# Percentiles a graficar (del 1 al 99)
percentiles = np.arange(1, 100)
valores_percentiles = [serie_percentil.quantile(p/100) for p in percentiles]

# Graficar la curva
fig_percentiles = go.Figure()
fig_percentiles.add_trace(go.Scatter(
    x=percentiles,
    y=valores_percentiles,
    mode='lines+markers',
    line=dict(color='#27AE60', width=3),
    marker=dict(size=6),
    name='Curva percentiles'
))

# Resalta P50, P75, P90
for px, clr, label in zip([50, 75, 90], ['#F4D03F', '#F39C12', '#C0392B'], ['P50', 'P75', 'P90']):
    val = serie_percentil.quantile(px/100)
    fig_percentiles.add_trace(go.Scatter(
        x=[px], y=[val],
        mode='markers+text',
        marker=dict(size=12, color=clr, symbol='diamond'),
        text=[label],
        textposition='top center',
        name=label
    ))

fig_percentiles.update_layout(
    title="Curva de percentiles de las diferencias (días)",
    xaxis_title="Percentil (%)",
    yaxis_title="Diferencia (días)",
    plot_bgcolor="white",
    font=dict(size=15),
    height=450
)

st.plotly_chart(fig_percentiles)
